# Trading Statistics Service - Historical Performance Metrics

Analyze trading statistics across multiple time periods (YTD, 1M, 3M, 6M, 1Y) including price, volume, and valuation metrics.

## Overview

The **Trading Statistics Service** provides multi-period historical data:

- **Time Periods**: YTD, 1 Month, 3 Months, 6 Months, 1 Year
- **Price Data**: OHLC (Open, High, Low, Close), average price
- **Volume Metrics**: Total volume, total value, average daily value
- **Valuation**: P/E, P/B, market cap, dividend yield
- **Risk**: Beta coefficient
- **Liquidity**: Turnover ratio

### When to Use

- Historical performance analysis
- Trend identification
- Volatility assessment
- Comparative period analysis
- Technical trading strategies

In [1]:
!pip install settfex

zsh:1: command not found: pip


In [2]:
import asyncio
from settfex.services.set import get_trading_stats
import pandas as pd
import matplotlib.pyplot as plt

print("Libraries imported!")

Libraries imported!


## Basic Usage

In [ ]:
# Fetch trading statistics
stats = await get_trading_stats("MINT")

print(f"Trading Statistics for MINT ({len(stats)} periods)\n")

for stat in stats:
    print(f"Period: {stat.period}")
    print(f"  Date: {stat.date}")
    print(f"  Close: {stat.close:.2f} THB ({stat.percent_change:+.2f}%)")
    print(f"  High: {stat.high:.2f}, Low: {stat.low:.2f}")
    print(f"  Volume: {stat.total_volume:,}")
    print(f"  P/E: {stat.pe}, Market Cap: {stat.market_cap:,.0f} THB")
    print()

## Advanced Usage - Performance Comparison

In [ ]:
# Convert to DataFrame for analysis
df = pd.DataFrame([
    {
        "period": s.period,
        "close": s.close,
        "change_pct": s.percent_change,
        "volume": s.total_volume,
        "avg_value": s.average_value,
        "pe": s.pe,
        "pbv": s.pbv,
        "beta": s.beta
    }
    for s in stats
])

print("Performance by Period:\n")
print(df[['period', 'close', 'change_pct', 'pe', 'beta']].to_string(index=False))

## Use Case: Multi-Stock Performance Dashboard

In [ ]:
async def performance_dashboard(symbols: list[str], period: str = "1M"):
    """
    Build performance dashboard for specific period.
    """
    tasks = [get_trading_stats(symbol) for symbol in symbols]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    dashboard = []
    for symbol, stats in zip(symbols, results):
        if isinstance(stats, Exception):
            continue
        
        # Find specific period
        period_stat = next((s for s in stats if s.period == period), None)
        if period_stat:
            dashboard.append({
                "symbol": symbol,
                "close": period_stat.close,
                "change_pct": period_stat.percent_change,
                "volume": period_stat.total_volume,
                "pe": period_stat.pe,
                "beta": period_stat.beta,
                "turnover": period_stat.turnover_ratio
            })
    
    df = pd.DataFrame(dashboard).sort_values('change_pct', ascending=False)
    return df

# Build dashboard
symbols = ["PTT", "KBANK", "CPALL", "AOT", "BBL", "SCB", "MINT"]
dashboard_1m = await performance_dashboard(symbols, period="1M")

print(f"1-Month Performance Dashboard:\n")
print(dashboard_1m.to_string(index=False))

## Use Case: Volatility Analysis

In [ ]:
# Compare volatility across periods
volatility_data = []
for stat in stats:
    if stat.high and stat.low and stat.close:
        price_range = ((stat.high - stat.low) / stat.close) * 100
        volatility_data.append({
            "period": stat.period,
            "price_range_pct": price_range,
            "beta": stat.beta
        })

vol_df = pd.DataFrame(volatility_data)
print("Volatility by Period:\n")
print(vol_df.to_string(index=False))

## Next Steps

- **[Highlight Data](02_highlight_data.ipynb)** - Current market metrics
- **[Price Performance](10_price_performance.ipynb)** - Comparative performance

**Documentation**: `/docs/settfex/services/set/trading_stat.md`